# Lab 3-1: REINFORCE
    In this lab, you need to implement a REINFORCE algorithm with Tensorflow and solve OpenAI Gym CartPole-v0

In [1]:
from cartpole_env import *

import numpy as np
import tensorflow as tf

from collections import namedtuple, deque

# Define the data structure of experience
Experience = namedtuple('Experience', 'state action reward next_state done')

## Implement ```discount``` function to compute discounted reward

In [2]:
def discount(rewards, gamma):
    '''
    param rewards: a rewards numpy array
    param gamma: discount factor
    '''
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(len(rewards))):
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

## Implement ```do_rollout``` function to collect rollout

In [3]:
def do_rollout(env, policy, render=False):
    '''
    Collect a rollout from env with policy
    
    param env: RL Environment
    param policy: a function parameterized by environment state, return a action
    return a list of (state, action, reward, next_state, done)
    '''
    # Initialize done as False
    done = False
    
    # Reset the environment and get the initial state
    state = env.reset()
    
    # Empty list
    rollout = []
    
    while not done:
        action = policy(state)
        next_state, reward, done, info = env.step(action)
        
        # Render the environment (slow)
        if render:
            env.render()
        
        rollout.append(Experience(state, action, reward, next_state, done))
        state = next_state
        
    return rollout

## Implement ```ReinforceAgent``` following ```TODO```

In [4]:
class ReinforceAgent(object):
    def __init__(self, sess, n_states, n_actions, n_hiddens, lr, gamma):
        '''
        param sess: tf session
        param n_states: dim of states
        param n_actions: dim of actions space
        param n_hiddens: dim of hidden state
        '''
        self.sess = sess
        self.n_states = n_states
        self.n_actions = n_actions
        
        # Learning rate
        self.lr = lr
        
        # Discount factor
        self.gamma = gamma
       
        self.state = tf.placeholder(shape=[None, n_states], dtype=tf.float32)
        self.value = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action = tf.placeholder(shape=[None], dtype=tf.int32)
        
        # Declare 1-st hidden layer
        # Define a fully-connected layer with:
        # input = self.state
        # n_units = self.n_hiddens
        # activation = relu
        # weight_initializer = random_normal(0.0, 0.1)
        # bias_initializer = constant (0.1)
        self.h = tf.layers.dense(
                inputs=self.state,
                units=n_hiddens,   
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1), 
                bias_initializer=tf.constant_initializer(0.1), 
                name='h')
        
        # Declare 1-st hidden layer
        # Define a fully-connected layer with:
        # input = 1-st hidden layer
        # n_units = self.n_actions
        # activation = relu
        # weight_initializer = random_normal(0.0, 0.1)
        # bias_initializer = constant (0.1)
        self.policy = tf.layers.dense(
                inputs=self.h,
                units=n_actions,    
                activation=tf.nn.softmax,  
                kernel_initializer=tf.random_normal_initializer(0., .1), 
                bias_initializer=tf.constant_initializer(0.1),
                name='policy')
        
        # negative log probability 
        neglog_prob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.policy, labels=self.action)
         
        # TODO: policy gradient loss function
        self.loss = tf.reduce_mean(neglog_prob * self.value) 
        
        # TODO: Optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        self.train_op = self.optimizer.minimize(self.loss)
        
    def act(self, s):
        '''
        param s: a np.ndarray with shape [n_batches, n_states]
        return a batch of actions with shape [n_batches,]
        '''
        policy = self.sess.run(self.policy, feed_dict={self.state: s})
        return np.array([np.random.choice(range(self.n_actions), p=p) for p in policy])
    
    def train(self, rollout):
        '''
        param rollout: a list o
        '''
        states = np.array([ np.asarray(e.state) for e in rollout ])
        actions = np.squeeze(np.array([ e.action for e in rollout ]))
        rewards = np.array([ e.reward for e in rollout ])
        discounted_rewards = discount(rewards, gamma=self.gamma)
        
        self.sess.run(self.train_op, feed_dict={self.state: states,
                                                self.action: actions,
                                                self.value: discounted_rewards})
   
        

In [5]:
LR = 0.001
GAMMA = 0.99

sess = tf.InteractiveSession()
env = CartpoleEnvironment()
agent = ReinforceAgent(sess=sess, 
                       n_states=env.observation_space.shape[0],
                       n_actions=env.action_space.n,
                       n_hiddens=20,
                       lr=LR,
                       gamma=GAMMA)
init = tf.global_variables_initializer()
sess.run(init)

[2017-10-16 18:24:02,574] Making new env: CartPole-v0


In [6]:
def policy(s):
    return agent.act([s])[0]

In [7]:
def calculate_episode_reward(rollout):
    rewards = [ e.reward for e in rollout ]
    return sum(rewards)

In [ ]:
def eval_history_reward(history):
    arr = np.asarray(history)
    return arr.mean()

In [ ]:
MAX_ITERATIONS = 100000

episode_reward = 0.0
history_episode_rewards = deque(maxlen=100)

plot_history_episode_rewards = []

for iter in range(MAX_ITERATIONS):
    rollout = do_rollout(env=env, policy=policy, render=False)
    agent.train(rollout=rollout)
    
    episode_reward = calculate_episode_reward(rollout)
    history_episode_rewards.append(episode_reward)
    plot_history_episode_rewards.append(episode_reward)
    mean_rewards = eval_history_reward(history_episode_rewards)
    
    print('Episode %d: Reward = %f, Mean reward (over %d episodes) = %f' % (iter, 
                                                                            episode_reward,
                                                                            len(history_episode_rewards),
                                                                            mean_rewards))
    if mean_rewards > 195.0:
        print('Pass')
        break

Episode 0: Reward = -6.000000, Mean reward (over 1 episodes) = -6.000000
Episode 1: Reward = -10.000000, Mean reward (over 2 episodes) = -8.000000
Episode 2: Reward = 2.000000, Mean reward (over 3 episodes) = -4.666667
Episode 3: Reward = 1.000000, Mean reward (over 4 episodes) = -3.250000
Episode 4: Reward = -8.000000, Mean reward (over 5 episodes) = -4.200000
Episode 5: Reward = -4.000000, Mean reward (over 6 episodes) = -4.166667
Episode 6: Reward = 17.000000, Mean reward (over 7 episodes) = -1.142857
Episode 7: Reward = -2.000000, Mean reward (over 8 episodes) = -1.250000
Episode 8: Reward = 4.000000, Mean reward (over 9 episodes) = -0.666667
Episode 9: Reward = -9.000000, Mean reward (over 10 episodes) = -1.500000
Episode 10: Reward = -7.000000, Mean reward (over 11 episodes) = -2.000000
Episode 11: Reward = -6.000000, Mean reward (over 12 episodes) = -2.333333
Episode 12: Reward = 75.000000, Mean reward (over 13 episodes) = 3.615385
Episode 13: Reward = -6.000000, Mean reward (ov

Episode 111: Reward = -10.000000, Mean reward (over 100 episodes) = 2.180000
Episode 112: Reward = 7.000000, Mean reward (over 100 episodes) = 1.500000
Episode 113: Reward = 9.000000, Mean reward (over 100 episodes) = 1.650000
Episode 114: Reward = 15.000000, Mean reward (over 100 episodes) = 1.840000
Episode 115: Reward = -9.000000, Mean reward (over 100 episodes) = 1.690000
Episode 116: Reward = 11.000000, Mean reward (over 100 episodes) = 1.570000
Episode 117: Reward = 24.000000, Mean reward (over 100 episodes) = 1.700000
Episode 118: Reward = -6.000000, Mean reward (over 100 episodes) = 1.740000
Episode 119: Reward = -4.000000, Mean reward (over 100 episodes) = 1.790000
Episode 120: Reward = 31.000000, Mean reward (over 100 episodes) = 1.970000
Episode 121: Reward = -2.000000, Mean reward (over 100 episodes) = 1.850000
Episode 122: Reward = -10.000000, Mean reward (over 100 episodes) = 1.790000
Episode 123: Reward = 4.000000, Mean reward (over 100 episodes) = 1.890000
Episode 124: 

Episode 229: Reward = 3.000000, Mean reward (over 100 episodes) = 5.780000
Episode 230: Reward = 55.000000, Mean reward (over 100 episodes) = 6.290000
Episode 231: Reward = 18.000000, Mean reward (over 100 episodes) = 6.410000
Episode 232: Reward = 0.000000, Mean reward (over 100 episodes) = 6.380000
Episode 233: Reward = -3.000000, Mean reward (over 100 episodes) = 6.340000
Episode 234: Reward = -8.000000, Mean reward (over 100 episodes) = 6.320000
Episode 235: Reward = -8.000000, Mean reward (over 100 episodes) = 5.410000
Episode 236: Reward = 5.000000, Mean reward (over 100 episodes) = 5.570000
Episode 237: Reward = 15.000000, Mean reward (over 100 episodes) = 5.610000
Episode 238: Reward = 38.000000, Mean reward (over 100 episodes) = 6.000000
Episode 239: Reward = -7.000000, Mean reward (over 100 episodes) = 5.790000
Episode 240: Reward = -4.000000, Mean reward (over 100 episodes) = 5.510000
Episode 241: Reward = 48.000000, Mean reward (over 100 episodes) = 6.030000
Episode 242: Re

Episode 339: Reward = 2.000000, Mean reward (over 100 episodes) = 10.090000
Episode 340: Reward = 2.000000, Mean reward (over 100 episodes) = 10.150000
Episode 341: Reward = 10.000000, Mean reward (over 100 episodes) = 9.770000
Episode 342: Reward = -5.000000, Mean reward (over 100 episodes) = 9.640000
Episode 343: Reward = 5.000000, Mean reward (over 100 episodes) = 9.770000
Episode 344: Reward = 23.000000, Mean reward (over 100 episodes) = 10.010000
Episode 345: Reward = -8.000000, Mean reward (over 100 episodes) = 9.840000
Episode 346: Reward = 28.000000, Mean reward (over 100 episodes) = 9.760000
Episode 347: Reward = -1.000000, Mean reward (over 100 episodes) = 9.360000
Episode 348: Reward = 69.000000, Mean reward (over 100 episodes) = 10.070000
Episode 349: Reward = 32.000000, Mean reward (over 100 episodes) = 10.440000
Episode 350: Reward = -7.000000, Mean reward (over 100 episodes) = 9.980000
Episode 351: Reward = 22.000000, Mean reward (over 100 episodes) = 9.790000
Episode 35

Episode 446: Reward = 32.000000, Mean reward (over 100 episodes) = 22.880000
Episode 447: Reward = 0.000000, Mean reward (over 100 episodes) = 22.890000
Episode 448: Reward = 58.000000, Mean reward (over 100 episodes) = 22.780000
Episode 449: Reward = 23.000000, Mean reward (over 100 episodes) = 22.690000
Episode 450: Reward = 44.000000, Mean reward (over 100 episodes) = 23.200000
Episode 451: Reward = 24.000000, Mean reward (over 100 episodes) = 23.220000
Episode 452: Reward = 34.000000, Mean reward (over 100 episodes) = 23.510000
Episode 453: Reward = 101.000000, Mean reward (over 100 episodes) = 24.590000
Episode 454: Reward = 34.000000, Mean reward (over 100 episodes) = 24.730000
Episode 455: Reward = 22.000000, Mean reward (over 100 episodes) = 24.890000
Episode 456: Reward = 22.000000, Mean reward (over 100 episodes) = 24.820000
Episode 457: Reward = 103.000000, Mean reward (over 100 episodes) = 25.930000
Episode 458: Reward = 75.000000, Mean reward (over 100 episodes) = 26.73000

...

Episode 26070: Reward = 154.000000, Mean reward (over 100 episodes) = 130.970000
Episode 26071: Reward = 139.000000, Mean reward (over 100 episodes) = 131.190000
Episode 26072: Reward = 134.000000, Mean reward (over 100 episodes) = 131.270000
Episode 26073: Reward = 119.000000, Mean reward (over 100 episodes) = 131.340000
Episode 26074: Reward = 148.000000, Mean reward (over 100 episodes) = 131.560000
Episode 26075: Reward = 115.000000, Mean reward (over 100 episodes) = 131.340000
Episode 26076: Reward = 146.000000, Mean reward (over 100 episodes) = 131.520000
Episode 26077: Reward = 122.000000, Mean reward (over 100 episodes) = 131.520000
Episode 26078: Reward = 145.000000, Mean reward (over 100 episodes) = 131.770000
Episode 26079: Reward = 131.000000, Mean reward (over 100 episodes) = 131.530000
Episode 26080: Reward = 136.000000, Mean reward (over 100 episodes) = 131.580000
Episode 26081: Reward = 129.000000, Mean reward (over 100 episodes) = 131.140000
Episode 26082: Reward = 167.

Episode 26174: Reward = 131.000000, Mean reward (over 100 episodes) = 131.900000
Episode 26175: Reward = 116.000000, Mean reward (over 100 episodes) = 131.910000
Episode 26176: Reward = 135.000000, Mean reward (over 100 episodes) = 131.800000
Episode 26177: Reward = 124.000000, Mean reward (over 100 episodes) = 131.820000
Episode 26178: Reward = 135.000000, Mean reward (over 100 episodes) = 131.720000
Episode 26179: Reward = 158.000000, Mean reward (over 100 episodes) = 131.990000
Episode 26180: Reward = 162.000000, Mean reward (over 100 episodes) = 132.250000
Episode 26181: Reward = 131.000000, Mean reward (over 100 episodes) = 132.270000
Episode 26182: Reward = 115.000000, Mean reward (over 100 episodes) = 131.750000
Episode 26183: Reward = 147.000000, Mean reward (over 100 episodes) = 132.000000
Episode 26184: Reward = 157.000000, Mean reward (over 100 episodes) = 132.060000
Episode 26185: Reward = 135.000000, Mean reward (over 100 episodes) = 131.950000
Episode 26186: Reward = 127.

Episode 26277: Reward = 136.000000, Mean reward (over 100 episodes) = 130.450000
Episode 26278: Reward = 141.000000, Mean reward (over 100 episodes) = 130.510000
Episode 26279: Reward = 131.000000, Mean reward (over 100 episodes) = 130.240000
Episode 26280: Reward = 117.000000, Mean reward (over 100 episodes) = 129.790000
Episode 26281: Reward = 122.000000, Mean reward (over 100 episodes) = 129.700000
Episode 26282: Reward = 132.000000, Mean reward (over 100 episodes) = 129.870000
Episode 26283: Reward = 117.000000, Mean reward (over 100 episodes) = 129.570000
Episode 26284: Reward = 128.000000, Mean reward (over 100 episodes) = 129.280000
Episode 26285: Reward = 117.000000, Mean reward (over 100 episodes) = 129.100000
Episode 26286: Reward = 130.000000, Mean reward (over 100 episodes) = 129.130000
Episode 26287: Reward = 138.000000, Mean reward (over 100 episodes) = 129.050000
Episode 26288: Reward = 121.000000, Mean reward (over 100 episodes) = 129.120000
Episode 26289: Reward = 108.

Episode 26381: Reward = 161.000000, Mean reward (over 100 episodes) = 130.640000
Episode 26382: Reward = 111.000000, Mean reward (over 100 episodes) = 130.430000
Episode 26383: Reward = 113.000000, Mean reward (over 100 episodes) = 130.390000
Episode 26384: Reward = 149.000000, Mean reward (over 100 episodes) = 130.600000
Episode 26385: Reward = 155.000000, Mean reward (over 100 episodes) = 130.980000
Episode 26386: Reward = 120.000000, Mean reward (over 100 episodes) = 130.880000
Episode 26387: Reward = 142.000000, Mean reward (over 100 episodes) = 130.920000
Episode 26388: Reward = 147.000000, Mean reward (over 100 episodes) = 131.180000
Episode 26389: Reward = 132.000000, Mean reward (over 100 episodes) = 131.420000
Episode 26390: Reward = 121.000000, Mean reward (over 100 episodes) = 131.030000
Episode 26391: Reward = 157.000000, Mean reward (over 100 episodes) = 131.110000
Episode 26392: Reward = 142.000000, Mean reward (over 100 episodes) = 131.300000
Episode 26393: Reward = 130.

Episode 26484: Reward = 170.000000, Mean reward (over 100 episodes) = 132.100000
Episode 26485: Reward = 125.000000, Mean reward (over 100 episodes) = 131.800000
Episode 26486: Reward = 124.000000, Mean reward (over 100 episodes) = 131.840000
Episode 26487: Reward = 106.000000, Mean reward (over 100 episodes) = 131.480000
Episode 26488: Reward = 123.000000, Mean reward (over 100 episodes) = 131.240000
Episode 26489: Reward = 128.000000, Mean reward (over 100 episodes) = 131.200000
Episode 26490: Reward = 124.000000, Mean reward (over 100 episodes) = 131.230000
Episode 26491: Reward = 104.000000, Mean reward (over 100 episodes) = 130.700000
Episode 26492: Reward = 103.000000, Mean reward (over 100 episodes) = 130.310000
Episode 26493: Reward = 136.000000, Mean reward (over 100 episodes) = 130.370000
Episode 26494: Reward = 105.000000, Mean reward (over 100 episodes) = 129.910000
Episode 26495: Reward = 143.000000, Mean reward (over 100 episodes) = 130.070000
Episode 26496: Reward = 121.

In [ ]:
import matplotlib.pyplot as plt

def plot(x, y, name):
    fig, ax = plt.subplots()
    ax.plot(x, y)
    
    ax.set(xlabel='Episode', ylabel='Reward', title=name)
    ax.grid()

    fig.savefig("%s.png" % name)
    plt.show()
    
plot(range(episode), plot_history_episode_rewards, 'REINFORCE')

In [ ]:
while True:
    do_rollout(env=env, policy=policy, render=True)